In [ ]:
#################################################################
# Use to generate data for Experiment 1
#################################################################


import pandas as pd
import numpy as np

df = pd.read_csv("cleaned_OI_state_data.csv")

#Generate Numerical Dates
def dates(df):
    def make_date(x):
        year = x[-4:]

        months = ['filler','jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug']
        a = {}
        a.update([(months[i],("0"+str(i))[-2:]) for i in range(1,len(months))])
        month=a[x[2:5]]

        day = x[:2]

        return year+month+day 

    df["mask_date"] = np.nan
    df["reopen_date"] = np.nan

    df.loc[df.mandate_mask.isnull()==False,"mask_date"]=df.loc[df.mandate_mask.isnull()==False, "mandate_mask"].apply(make_date)
    df.loc[df.begin_reopen.isnull()==False,"reopen_date"]=df.loc[df.begin_reopen.isnull()==False,"begin_reopen"].apply(make_date)
    df["date_num"] = df["date"].apply(make_date)
    
    
    df["mask_after"]=np.nan
    df["after_reopen"]=0
    for i in df.loc[df.mandate_mask.isnull()==False].index:
        if df["mask_date"][i]>df["reopen_date"][i]: df["mask_after"][i]=1
        else: df["mask_after"][i]=0
            
        if df["date_num"][i]>=df["reopen_date"][i]: df["after_reopen"][i]=1

#Generate an indicator variable for each state
def state_dummy(df):
    sas = df.state_abbrev.unique()
    for i in sas:
        df["in"+i]=0
        df.loc[df.state_abbrev==i, "in"+i]=1

#Generate time variant fixed effect dummies- one for every half month
def half_month_dummy(df):
    for i in df.index:
        if int(df["date"][i][:2])<=15:
            half="1"
        else: half="2"
        mon = df["date"][i][2:5]
        
        col = "hmon_"+mon+half
        if col not in df.columns:
            df[col]=0
        
        df[col][i]=1

#Generate time variant fixed effect dummies- one for every month
def month_dummy(df):
    for i in df.index:
        mon = df["date"][i][2:5]
        
        col = "mon_"+mon
        if col not in df.columns:
            df[col]=0
        
        df[col][i]=1

#Generate timevar variable for eventdd
#timevar = i if it is i days after state s has implemented the mandate
#i.e. timevar = -1 one day before mandate
#i.e. timevar = 0 on day of mandate
#i.e. timevar = 1 one day after mandate
def gen_timevar(df):
    df["timevar"]=0

    cstate = ""
    counter = 0
    started = False
    for i in df.index:
        if df["state"][i]!=cstate: 
            counter = 0
            cstate = df["state"][i]
            started = False

        if df["mandate_mask"][i]==df["date"][i]:
            started = True

        if started:
            df["timevar"][i]=counter
            counter+=1

    back = list(df.index).copy()
    back.reverse()
    back

    cstate = ""
    counter = 0
    started = False
    for i in back:
        if df["state"][i]!=cstate: 
            counter = 0
            cstate = df["state"][i]
            started = False

        if df["mandate_mask"][i]==df["date"][i]:
            started = True

        if started:
            df["timevar"][i]=counter
            counter-=1
    
#Merge a column from COVID case dataset into df
def merge_col(df, file, column):
    rev = pd.read_csv(file)
    rev = rev.loc[rev.month<9]
    
    def make_ind_rev(x):
        day = ("0"+str(x["day"]))[-2:]
        month = ['filler','jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug'][x["month"]]

        return day+month+str(x["year"])+"/"+str(x["statefips"])

    def make_ind_df(x):
        return x["date"]+"/"+str(x["statefips"])


    rev["ind"] = rev.apply(make_ind_rev, axis=1)
    df["ind"]  = df.apply(make_ind_df, axis=1)

    df = df.join(rev.set_index('ind')[column], on='ind', how="inner")
    
    return df

#Get the daily percent change in col for each state
def get_pctchange(df, col):
    df["pctch"+col]=0.0
    for i in df.state.unique(): 
        last = df.loc[df.state==i,col].shift(1)
        curr = df.loc[df.state==i,col]
        res = 100*(np.log(curr)-np.log(last))
        res = res.fillna(0)
        res[res==np.inf]=0
        for j in res.index:
            df["pctch"+col][j]=res[j]
            
#Group timevar into 20 day periods
def group(df):
    df["tv2"]=0
    df.loc[df.timevar<-90, "tv2"] = -4
    df.loc[(df.timevar<-50) & (df.timevar>=-70), "tv2"] = -3
    df.loc[(df.timevar<-30) & (df.timevar>=-50), "tv2"] = -2
    df.loc[(df.timevar<-10) & (df.timevar>=-30), "tv2"] = -1
    df.loc[(df.timevar>-10) & (df.timevar<=10), "tv2"] = 0
    df.loc[(df.timevar>10) & (df.timevar<=30), "tv2"] = 1
    df.loc[(df.timevar>30) & (df.timevar<=50), "tv2"] = 2
    df.loc[(df.timevar>50) & (df.timevar<=70), "tv2"] = 3
    df.loc[(df.timevar>70) & (df.timevar<=90), "tv2"] = 4
    df.loc[(df.timevar>90) & (df.timevar<=110), "tv2"] = 5
    df.loc[(df.timevar>110) & (df.timevar<=130), "tv2"] = 6
    df.loc[(df.timevar>130), "tv2"] = 7
    
            

gen_timevar(df)
gen_lags(df)
state_dummy(df)
half_month_dummy(df)
month_dummy(df)
dates(df)
group(df)
df = merge_col(df,"revenue.csv", "revenue_all")
df = merge_col(df,"covid.csv", "new_case_count")
df = merge_col(df,"covid.csv", "new_case_rate")
df = merge_col(df,"covid.csv", "new_test_rate")kkk

df = merge_col(df,"covid.csv", "case_count")
df = merge_col(df,"covid.csv", "test_count")

get_pctchange(df, "case_count")

df.loc[df.test_count==".", "test_count"] = 0
df["test_count"] = df["test_count"].apply(lambda x: int(x))
get_pctchange(df, "test_count")

df = df.drop(["ind"], axis=1)
df.to_csv("masks2.csv")

In [ ]:
#################################################################
# Use to generate data for Experiment 1
#################################################################


#Generate and save the matrix with compliance data
def gen_compliance():
    comp = pd.read_csv("~/Downloads/mask_compliance.csv")
    ctylevel = pd.read_csv("~/Downloads/covidcounty.csv")

    #add county level case counts to compliance dataframe
    ctylevel.loc[ctylevel.new_case_count=="."]=0
    ctylevel["new_case_count"] = ctylevel["new_case_count"].apply(lambda x: int(x))
    total_cases = ctylevel.loc[ctylevel.month>=7,["new_case_count","countyfips"]].groupby("countyfips").sum()
    comp = comp.join(total_cases["new_case_count"], on='COUNTYFP', how="inner").dropna()

    #add number of cases in the last month per county
    recent_cases = ctylevel.loc[ctylevel.month==6,["new_case_count","countyfips"]].groupby("countyfips").sum()
    recent_cases.columns = ["recent_cases"]
    comp = comp.join(recent_cases["recent_cases"], on='COUNTYFP', how="inner").dropna()

    #add cumulative cases from start of data to mask mandate date
    ctylevel["case_count"] = ctylevel["case_count"].apply(lambda x: int(x))
    cases_til = ctylevel.loc[(ctylevel.month==7) & (ctylevel.day==1)]
    comp = comp.join(cases_til.set_index("countyfips")["case_count"], on='COUNTYFP', how="inner")

    #Add county level population numbers from census bureau data set
    pops = pd.read_csv("~/Downloads/co-est2019-alldata.csv", usecols=["STATE","COUNTY","STNAME","POPESTIMATE2019"])
    def fipscode(x):
        county = ("00"+str(x["COUNTY"]))[-3:]
        state = str(x["STATE"])
        return int(state+county)
    pops["fipscode"] = pops.apply(fipscode, axis=1)

    print(len(pops))

    #Add state name and county level population
    comp = comp.join(pops.set_index('fipscode')["POPESTIMATE2019"], on='COUNTYFP', how="inner")
    comp = comp.join(pops.set_index('fipscode')["STNAME"], on='COUNTYFP', how="inner")

    #normalize covid data per 10,000 people
    comp["new_case_rate"]=comp["new_case_count"]/comp["POPESTIMATE2019"]*10_000
    comp["case_count_rate"]=comp["case_count"]/comp["POPESTIMATE2019"]*10_000
    comp["recent_case_rate"]=comp["recent_cases"]/comp["POPESTIMATE2019"]*10_000

    #add mask dummy
    comp["mask_mandate"] = 0
    for i in df.loc[df.mask_date.isnull()==False, "state"].unique():
        mask_date = df.loc[df.state == i, "mask_date"].unique()[0]
        if int(mask_date)<20200702: comp.loc[comp.STNAME==i,"mask_mandate"]=1

    #add reopened dummy
    comp["reopened"] = 0
    for i in df.loc[df.reopen_date.isnull()==False, "state"].unique():
        reopen_date = df.loc[df.state == i, "reopen_date"].unique()[0]
        if int(reopen_date)<20200702: comp.loc[comp.STNAME==i,"reopened"]=1

    #generate target variable for regression
    comp["atleastFreq"] = comp["FREQUENTLY"]+comp["ALWAYS"]

    #add dummy for republican governor
    comp["gov_rep"] = 0
    for i in comp.STNAME.unique():
        if df.loc[df.state==i,"governor_party"].unique()[0]=="Republican": comp.loc[comp.STNAME==i,"gov_rep"]=1 

    #save data
    comp.to_csv("compliance_ds.csv")
    
gen_compliance()